In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values #no need for rowid, customer and others as they have no value in outcome
y = dataset.iloc[:, 13].values
print(X)
print(y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
[1 0 1 ... 1 1 0]


In [2]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1]) #label encode country
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
print(X)
#create dummy variables for countries as one country is not above another
onehotencoder = OneHotEncoder(categorical_features = [1]) #[1] index for which dummy variable is needed
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:] #remove one dummy variable to avoid dummy trap
print(X)

[[619 0 0 ... 1 1 101348.88]
 [608 2 0 ... 0 1 112542.58]
 [502 0 0 ... 1 0 113931.57]
 ...
 [709 0 0 ... 0 1 42085.58]
 [772 1 1 ... 1 0 92888.52]
 [792 0 0 ... 1 0 38190.78]]
[[0.0000000e+00 0.0000000e+00 6.1900000e+02 ... 1.0000000e+00
  1.0000000e+00 1.0134888e+05]
 [0.0000000e+00 1.0000000e+00 6.0800000e+02 ... 0.0000000e+00
  1.0000000e+00 1.1254258e+05]
 [0.0000000e+00 0.0000000e+00 5.0200000e+02 ... 1.0000000e+00
  0.0000000e+00 1.1393157e+05]
 ...
 [0.0000000e+00 0.0000000e+00 7.0900000e+02 ... 0.0000000e+00
  1.0000000e+00 4.2085580e+04]
 [1.0000000e+00 0.0000000e+00 7.7200000e+02 ... 1.0000000e+00
  0.0000000e+00 9.2888520e+04]
 [0.0000000e+00 0.0000000e+00 7.9200000e+02 ... 1.0000000e+00
  0.0000000e+00 3.8190780e+04]]


In [3]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [4]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [5]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
# Initialize weights to random number 
# no rule for output dim experiments is one way another is average of number of variables in input and output
# so here 11+1 = 12/2
# init is weight assignment here uniform is simplest which assigns close to zero numbers
# activation function for this hidden layer
# input dim is input variables here 11

classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = 11))
# to reduce overfitting through this some neurons are disabled random to avoid dependency
# can be added to multiple layers
classifier.add(Dropout(p = 0.1))
# Adding the second hidden layer
# no input dim as it is only for first layer
# here to activation is rectifier
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))

# Adding the output layer
# output dim is 1 as output is 1
# activation is sigmoid as as we need probabilities for the customer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
# optimizer work is to select the weights. we choose stocashtic gradient decent and one of that is adam
# loss function is used to optimize the weights like one is squared to difference of actual and predicted value
# is output is binary then loss is binary_crossentropy if two variable output then categorical_crossentropy
# we choose metric accuracy as we need accuracy to be improved, it expects list that's why ['accuracy']

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
# input training set
# batch size after which we want to update weights
# 1 epoch is when whole training set is passed around
classifier.fit(X_train, y_train, batch_size = 10, nb_epoch = 100)

'''
In case of regression task the activation on initial will be same i.e. relu as it is an rectifier
and only activates above certain levels.
In last layer the activation will be linear and loss will be mean absolute error instead of binary ones.
'''

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/home/bhawesh/.local/lib/python3.5/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", input_dim=11, units=6, activation="relu")`
/home/bhawesh/.local/lib/python3.5/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", units=6, activation="relu")`
/home/bhawesh/.local/lib/python3.5/site-packages/ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", units=1, activation="sigmoid")`
/home/bhawesh/.local/lib/python3.5/site-packages/ipykernel_launcher.py:41: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/100
8000/8000 [==============================] - 2s 268us/step - loss: 0.4904 - acc: 0.7954
Epoch 2/100
8000/8000 [==============================] - 1s 105us/step - loss: 0.4270 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 105us/step - loss: 0.4205 - acc: 0.8236
Epoch 4/100
8000/8000 [==============================] - 1s 105us/step - loss: 0.4140 - acc: 0.8304
Epoch 5/100
8000/8000 [==============================] - 1s 103us/step - loss: 0.4092 - acc: 0.8330
Epoch 6/100
8000/8000 [==============================] - 1s 104us/step - loss: 0.4057 - acc: 0.8324
Epoch 7/100
8000/8000 [==============================] - 1s 104us/step - loss: 0.4041 - acc: 0.8342
Epoch 8/100
8000/8000 [==============================] - 1s 106us/step - loss: 0.4023 - acc: 0.8342
Epoch 9/100
8000/8000 [==============================] - 1s 106us/step - loss: 0.4015 - acc: 0.8361
Epoch 10/100
8000/8000 [==============================] - 1s 107us/step - loss: 0.4002 - acc: 0.8349

8000/8000 [==============================] - 1s 106us/step - loss: 0.3317 - acc: 0.8685
Epoch 83/100
8000/8000 [==============================] - 1s 106us/step - loss: 0.3317 - acc: 0.8657
Epoch 84/100
8000/8000 [==============================] - 1s 111us/step - loss: 0.3318 - acc: 0.8655
Epoch 85/100
8000/8000 [==============================] - 1s 106us/step - loss: 0.3319 - acc: 0.8630
Epoch 86/100
8000/8000 [==============================] - 1s 107us/step - loss: 0.3320 - acc: 0.8650
Epoch 87/100
8000/8000 [==============================] - 1s 108us/step - loss: 0.3317 - acc: 0.8637
Epoch 88/100
8000/8000 [==============================] - 1s 105us/step - loss: 0.3311 - acc: 0.8660
Epoch 89/100
8000/8000 [==============================] - 1s 113us/step - loss: 0.3303 - acc: 0.8664
Epoch 90/100
8000/8000 [==============================] - 1s 126us/step - loss: 0.3300 - acc: 0.8667
Epoch 91/100
8000/8000 [==============================] - 1s 107us/step - loss: 0.3300 - acc: 0.8670
Epo

'\nIn case of regression task the activation on initial will be same i.e. relu as it is an rectifier\nand only activates above certain levels.\nIn last layer the activation will be linear and loss will be mean absolute error instead of binary ones.\n'

In [7]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

# this is done so output is true or false so we set a threshhold for it
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [8]:
accuracy = (cm[0][0] + cm[1][1]) / 2000
print(accuracy)

0.856


In [32]:
# dataset.iloc[2,3:13].values.tolist()
# single prediction
classifier.predict(np.array( [[0.23, 1, 0, 0, .42, 8, .1596608, 3, 1, 0, .11393157],] ))

array([[0.24806967]], dtype=float32)

In [11]:
# frace dummy variable set is 0,0
new_prediction = classifier.predict(sc.transform(np.array([[0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction

/home/bhawesh/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [16]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)

# cross validation 10 this will divide train set in 10 part and in each part there is a minor test set
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = 1)
mean = accuracies.mean()
variance = accuracies.std()
print(mean, variance)


Epoch 1/100
7200/7200 [==============================] - 1s 156us/step - loss: 0.4961 - acc: 0.7961
Epoch 2/100
7200/7200 [==============================] - 1s 111us/step - loss: 0.4276 - acc: 0.7971
Epoch 3/100
7200/7200 [==============================] - 1s 186us/step - loss: 0.4237 - acc: 0.7971
Epoch 4/100
7200/7200 [==============================] - 1s 172us/step - loss: 0.4194 - acc: 0.8001
Epoch 5/100
7200/7200 [==============================] - 1s 177us/step - loss: 0.4172 - acc: 0.8254
Epoch 6/100
7200/7200 [==============================] - 1s 184us/step - loss: 0.4152 - acc: 0.8286
Epoch 7/100
7200/7200 [==============================] - 1s 189us/step - loss: 0.4139 - acc: 0.8290
Epoch 8/100
7200/7200 [==============================] - 2s 225us/step - loss: 0.4127 - acc: 0.8311 1s - loss: 0.42
Epoch 9/100
7200/7200 [==============================] - 1s 179us/step - loss: 0.4117 - acc: 0.8307
Epoch 10/100
7200/7200 [==============================] - 1s 174us/step - loss: 0.41

7200/7200 [==============================] - 1s 107us/step - loss: 0.4006 - acc: 0.8343
Epoch 82/100
7200/7200 [==============================] - 1s 102us/step - loss: 0.4010 - acc: 0.8346
Epoch 83/100
7200/7200 [==============================] - 1s 101us/step - loss: 0.4008 - acc: 0.8349
Epoch 84/100
7200/7200 [==============================] - 1s 102us/step - loss: 0.4007 - acc: 0.8343
Epoch 85/100
7200/7200 [==============================] - 1s 102us/step - loss: 0.4006 - acc: 0.8353
Epoch 86/100
7200/7200 [==============================] - 1s 103us/step - loss: 0.3999 - acc: 0.8344
Epoch 87/100
7200/7200 [==============================] - 1s 101us/step - loss: 0.4001 - acc: 0.8349
Epoch 88/100
7200/7200 [==============================] - 1s 103us/step - loss: 0.4009 - acc: 0.8350
Epoch 89/100
7200/7200 [==============================] - 1s 102us/step - loss: 0.4007 - acc: 0.8344
Epoch 90/100
7200/7200 [==============================] - 1s 102us/step - loss: 0.4004 - acc: 0.8346
Epo

7200/7200 [==============================] - 1s 131us/step - loss: 0.4004 - acc: 0.8354
Epoch 62/100
7200/7200 [==============================] - 1s 123us/step - loss: 0.4009 - acc: 0.8358
Epoch 63/100
7200/7200 [==============================] - 1s 112us/step - loss: 0.4006 - acc: 0.8353
Epoch 64/100
7200/7200 [==============================] - 1s 124us/step - loss: 0.4006 - acc: 0.8354
Epoch 65/100
7200/7200 [==============================] - 1s 125us/step - loss: 0.4003 - acc: 0.8349
Epoch 66/100
7200/7200 [==============================] - 1s 124us/step - loss: 0.4005 - acc: 0.8360
Epoch 67/100
7200/7200 [==============================] - 1s 139us/step - loss: 0.4003 - acc: 0.8365
Epoch 68/100
7200/7200 [==============================] - 1s 120us/step - loss: 0.4001 - acc: 0.8347
Epoch 69/100
7200/7200 [==============================] - 1s 153us/step - loss: 0.4002 - acc: 0.8350
Epoch 70/100
7200/7200 [==============================] - 1s 130us/step - loss: 0.3999 - acc: 0.8354
Epo

7200/7200 [==============================] - 1s 122us/step - loss: 0.4025 - acc: 0.8353
Epoch 42/100
7200/7200 [==============================] - 1s 126us/step - loss: 0.4016 - acc: 0.8351
Epoch 43/100
7200/7200 [==============================] - 1s 124us/step - loss: 0.4016 - acc: 0.8344
Epoch 44/100
7200/7200 [==============================] - 1s 118us/step - loss: 0.4026 - acc: 0.8354
Epoch 45/100
7200/7200 [==============================] - 1s 122us/step - loss: 0.4022 - acc: 0.8353
Epoch 46/100
7200/7200 [==============================] - 1s 125us/step - loss: 0.4018 - acc: 0.8335
Epoch 47/100
7200/7200 [==============================] - 1s 124us/step - loss: 0.4020 - acc: 0.8362
Epoch 48/100
7200/7200 [==============================] - 1s 125us/step - loss: 0.4012 - acc: 0.8337
Epoch 49/100
7200/7200 [==============================] - 1s 126us/step - loss: 0.4020 - acc: 0.8337
Epoch 50/100
7200/7200 [==============================] - 1s 126us/step - loss: 0.4012 - acc: 0.8356
Epo

7200/7200 [==============================] - 1s 120us/step - loss: 0.4028 - acc: 0.8350
Epoch 22/100
7200/7200 [==============================] - 1s 124us/step - loss: 0.4020 - acc: 0.8351
Epoch 23/100
7200/7200 [==============================] - 1s 122us/step - loss: 0.4024 - acc: 0.8357
Epoch 24/100
7200/7200 [==============================] - 1s 121us/step - loss: 0.4012 - acc: 0.8339
Epoch 25/100
7200/7200 [==============================] - 1s 118us/step - loss: 0.4016 - acc: 0.8351
Epoch 26/100
7200/7200 [==============================] - 1s 123us/step - loss: 0.4016 - acc: 0.8339
Epoch 27/100
7200/7200 [==============================] - 1s 123us/step - loss: 0.4013 - acc: 0.8360
Epoch 28/100
7200/7200 [==============================] - 1s 126us/step - loss: 0.4008 - acc: 0.8349
Epoch 29/100
7200/7200 [==============================] - 1s 122us/step - loss: 0.4009 - acc: 0.8343
Epoch 30/100
7200/7200 [==============================] - 1s 123us/step - loss: 0.4008 - acc: 0.8361
Epo

...

7200/7200 [==============================] - 1s 194us/step - loss: 0.3966 - acc: 0.8349
Epoch 43/100
7200/7200 [==============================] - 1s 204us/step - loss: 0.3966 - acc: 0.8354
Epoch 44/100
7200/7200 [==============================] - 1s 203us/step - loss: 0.3963 - acc: 0.8344
Epoch 45/100
7200/7200 [==============================] - 1s 196us/step - loss: 0.3960 - acc: 0.8354
Epoch 46/100
7200/7200 [==============================] - 1s 202us/step - loss: 0.3959 - acc: 0.8356
Epoch 47/100
7200/7200 [==============================] - 1s 207us/step - loss: 0.3965 - acc: 0.8360
Epoch 48/100
7200/7200 [==============================] - 1s 204us/step - loss: 0.3955 - acc: 0.8372
Epoch 49/100
7200/7200 [==============================] - 1s 208us/step - loss: 0.3954 - acc: 0.8374
Epoch 50/100
7200/7200 [==============================] - 2s 216us/step - loss: 0.3954 - acc: 0.8368
Epoch 51/100
7200/7200 [==============================] - 1s 206us/step - loss: 0.3956 - acc: 0.8367
Epo

7200/7200 [==============================] - 1s 202us/step - loss: 0.3977 - acc: 0.8356
Epoch 23/100
7200/7200 [==============================] - 1s 207us/step - loss: 0.3964 - acc: 0.8354
Epoch 24/100
7200/7200 [==============================] - 1s 202us/step - loss: 0.3968 - acc: 0.8360
Epoch 25/100
7200/7200 [==============================] - 1s 191us/step - loss: 0.3964 - acc: 0.8365
Epoch 26/100
7200/7200 [==============================] - 1s 188us/step - loss: 0.3959 - acc: 0.8358
Epoch 27/100
7200/7200 [==============================] - 1s 205us/step - loss: 0.3959 - acc: 0.8369
Epoch 28/100
7200/7200 [==============================] - 1s 197us/step - loss: 0.3956 - acc: 0.8351
Epoch 29/100
7200/7200 [==============================] - 1s 183us/step - loss: 0.3955 - acc: 0.8379
Epoch 30/100
7200/7200 [==============================] - 1s 190us/step - loss: 0.3954 - acc: 0.8364
Epoch 31/100
7200/7200 [==============================] - 1s 189us/step - loss: 0.3948 - acc: 0.8368
Epo

7200/7200 [==============================] - 1s 110us/step - loss: 0.4173 - acc: 0.7957
Epoch 3/100
7200/7200 [==============================] - 1s 119us/step - loss: 0.4093 - acc: 0.7957
Epoch 4/100
7200/7200 [==============================] - 1s 109us/step - loss: 0.3998 - acc: 0.8228
Epoch 5/100
7200/7200 [==============================] - 1s 104us/step - loss: 0.3910 - acc: 0.8282
Epoch 6/100
7200/7200 [==============================] - 1s 139us/step - loss: 0.3835 - acc: 0.8293
Epoch 7/100
7200/7200 [==============================] - 1s 154us/step - loss: 0.3778 - acc: 0.8325
Epoch 8/100
7200/7200 [==============================] - 1s 168us/step - loss: 0.3735 - acc: 0.8433
Epoch 9/100
7200/7200 [==============================] - 1s 149us/step - loss: 0.3693 - acc: 0.8474
Epoch 10/100
7200/7200 [==============================] - 1s 118us/step - loss: 0.3664 - acc: 0.8521
Epoch 11/100
7200/7200 [==============================] - 1s 154us/step - loss: 0.3639 - acc: 0.8500
Epoch 12/1

7200/7200 [==============================] - 1s 109us/step - loss: 0.3403 - acc: 0.8628
Epoch 84/100
7200/7200 [==============================] - 1s 110us/step - loss: 0.3406 - acc: 0.8625
Epoch 85/100
7200/7200 [==============================] - 1s 108us/step - loss: 0.3403 - acc: 0.8612
Epoch 86/100
7200/7200 [==============================] - 1s 112us/step - loss: 0.3391 - acc: 0.8625
Epoch 87/100
7200/7200 [==============================] - 1s 109us/step - loss: 0.3395 - acc: 0.8622
Epoch 88/100
7200/7200 [==============================] - 1s 112us/step - loss: 0.3398 - acc: 0.8606
Epoch 89/100
7200/7200 [==============================] - 1s 110us/step - loss: 0.3398 - acc: 0.8624
Epoch 90/100
7200/7200 [==============================] - 1s 126us/step - loss: 0.3407 - acc: 0.8600
Epoch 91/100
7200/7200 [==============================] - 1s 138us/step - loss: 0.3393 - acc: 0.8626
Epoch 92/100
7200/7200 [==============================] - 1s 115us/step - loss: 0.3402 - acc: 0.8597
Epo

7200/7200 [==============================] - 1s 137us/step - loss: 0.4035 - acc: 0.8343
Epoch 64/100
7200/7200 [==============================] - 1s 121us/step - loss: 0.4033 - acc: 0.8322
Epoch 65/100
7200/7200 [==============================] - 1s 138us/step - loss: 0.4044 - acc: 0.8324
Epoch 66/100
7200/7200 [==============================] - 1s 133us/step - loss: 0.4039 - acc: 0.8333
Epoch 67/100
7200/7200 [==============================] - 1s 146us/step - loss: 0.4041 - acc: 0.8351
Epoch 68/100
7200/7200 [==============================] - 1s 133us/step - loss: 0.4038 - acc: 0.8329
Epoch 69/100
7200/7200 [==============================] - 1s 122us/step - loss: 0.4040 - acc: 0.8333
Epoch 70/100
7200/7200 [==============================] - 1s 123us/step - loss: 0.4039 - acc: 0.8337
Epoch 71/100
7200/7200 [==============================] - 1s 127us/step - loss: 0.4038 - acc: 0.8342
Epoch 72/100
7200/7200 [==============================] - 1s 127us/step - loss: 0.4037 - acc: 0.8318
Epo

In [17]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed
# through dropout during training random neurons are switched off so that it doesn't form a dependent relation
# 
# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)

# hyperparameters we want to try and figure out the best one
parameters = {'batch_size': [25, 32],
              'epochs': [100, 500],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/100
7200/7200 [==============================] - 2s 288us/step - loss: 0.5563 - acc: 0.7965
Epoch 2/100
7200/7200 [==============================] - 1s 97us/step - loss: 0.4355 - acc: 0.7971
Epoch 3/100
7200/7200 [==============================] - 1s 125us/step - loss: 0.4314 - acc: 0.7971
Epoch 4/100
7200/7200 [==============================] - 1s 112us/step - loss: 0.4289 - acc: 0.7971
Epoch 5/100
7200/7200 [==============================] - 1s 99us/step - loss: 0.4274 - acc: 0.7971
Epoch 6/100
7200/7200 [==============================] - 1s 107us/step - loss: 0.4255 - acc: 0.7971
Epoch 7/100
7200/7200 [==============================] - 1s 106us/step - loss: 0.4231 - acc: 0.7971
Epoch 8/100
7200/7200 [==============================] - 1s 107us/step - loss: 0.4209 - acc: 0.8122
Epoch 9/100
7200/7200 [==============================] - 1s 112us/step - loss: 0.4189 - acc: 0.8187
Epoch 10/100
7200/7200 [==============================] - 1s 103us/step - loss: 0.4178 - acc: 0.8226
E

7200/7200 [==============================] - 1s 93us/step - loss: 0.4005 - acc: 0.8347
Epoch 83/100
7200/7200 [==============================] - 1s 117us/step - loss: 0.4000 - acc: 0.8360
Epoch 84/100
7200/7200 [==============================] - 1s 105us/step - loss: 0.4001 - acc: 0.8343
Epoch 85/100
7200/7200 [==============================] - 1s 103us/step - loss: 0.4000 - acc: 0.8347
Epoch 86/100
7200/7200 [==============================] - 1s 100us/step - loss: 0.4001 - acc: 0.8351
Epoch 87/100
7200/7200 [==============================] - 1s 97us/step - loss: 0.4000 - acc: 0.8362
Epoch 88/100
7200/7200 [==============================] - 1s 95us/step - loss: 0.4003 - acc: 0.8353
Epoch 89/100
7200/7200 [==============================] - 1s 103us/step - loss: 0.4000 - acc: 0.8353
Epoch 90/100
7200/7200 [==============================] - 1s 104us/step - loss: 0.4002 - acc: 0.8356
Epoch 91/100
7200/7200 [==============================] - 1s 109us/step - loss: 0.4000 - acc: 0.8336
Epoch 

7200/7200 [==============================] - 0s 62us/step - loss: 0.3337 - acc: 0.8649
Epoch 64/100
7200/7200 [==============================] - 0s 63us/step - loss: 0.3337 - acc: 0.8635
Epoch 65/100
7200/7200 [==============================] - 0s 51us/step - loss: 0.3337 - acc: 0.8654
Epoch 66/100
7200/7200 [==============================] - 0s 64us/step - loss: 0.3333 - acc: 0.8622
Epoch 67/100
7200/7200 [==============================] - 1s 73us/step - loss: 0.3336 - acc: 0.8665
Epoch 68/100
7200/7200 [==============================] - 1s 113us/step - loss: 0.3335 - acc: 0.8650
Epoch 69/100
7200/7200 [==============================] - 1s 89us/step - loss: 0.3332 - acc: 0.8660
Epoch 70/100
7200/7200 [==============================] - 1s 106us/step - loss: 0.3337 - acc: 0.8637
Epoch 71/100
7200/7200 [==============================] - 1s 102us/step - loss: 0.3334 - acc: 0.8653
Epoch 72/100
7200/7200 [==============================] - 1s 96us/step - loss: 0.3335 - acc: 0.8646
Epoch 73/1

7200/7200 [==============================] - 1s 98us/step - loss: 0.4034 - acc: 0.8336
Epoch 45/100
7200/7200 [==============================] - 1s 105us/step - loss: 0.4029 - acc: 0.8358
Epoch 46/100
7200/7200 [==============================] - 1s 107us/step - loss: 0.4030 - acc: 0.8347
Epoch 47/100
7200/7200 [==============================] - 1s 101us/step - loss: 0.4028 - acc: 0.8371
Epoch 48/100
7200/7200 [==============================] - 1s 91us/step - loss: 0.4029 - acc: 0.8350
Epoch 49/100
7200/7200 [==============================] - 1s 94us/step - loss: 0.4031 - acc: 0.8356
Epoch 50/100
7200/7200 [==============================] - 1s 106us/step - loss: 0.4024 - acc: 0.8361
Epoch 51/100
7200/7200 [==============================] - 1s 112us/step - loss: 0.4025 - acc: 0.8351
Epoch 52/100
7200/7200 [==============================] - 1s 90us/step - loss: 0.4024 - acc: 0.8354
Epoch 53/100
7200/7200 [==============================] - 1s 99us/step - loss: 0.4025 - acc: 0.8356
Epoch 54

7200/7200 [==============================] - 0s 63us/step - loss: 0.3937 - acc: 0.8372
Epoch 26/100
7200/7200 [==============================] - 0s 65us/step - loss: 0.3940 - acc: 0.8367
Epoch 27/100
7200/7200 [==============================] - 0s 63us/step - loss: 0.3934 - acc: 0.8374
Epoch 28/100
7200/7200 [==============================] - 0s 56us/step - loss: 0.3934 - acc: 0.8381
Epoch 29/100
7200/7200 [==============================] - 1s 71us/step - loss: 0.3927 - acc: 0.8374
Epoch 30/100
7200/7200 [==============================] - 0s 56us/step - loss: 0.3928 - acc: 0.8378
Epoch 31/100
7200/7200 [==============================] - 0s 64us/step - loss: 0.3924 - acc: 0.8379
Epoch 32/100
7200/7200 [==============================] - 0s 62us/step - loss: 0.3925 - acc: 0.8381
Epoch 33/100
7200/7200 [==============================] - 0s 64us/step - loss: 0.3922 - acc: 0.8382
Epoch 34/100
7200/7200 [==============================] - 0s 64us/step - loss: 0.3922 - acc: 0.8365
Epoch 35/100


...

7200/7200 [==============================] - 1s 93us/step - loss: 0.3973 - acc: 0.8349
Epoch 89/100
7200/7200 [==============================] - 1s 100us/step - loss: 0.3971 - acc: 0.8332
Epoch 90/100
7200/7200 [==============================] - 1s 122us/step - loss: 0.3974 - acc: 0.8344
Epoch 91/100
7200/7200 [==============================] - 1s 117us/step - loss: 0.3971 - acc: 0.8351
Epoch 92/100
7200/7200 [==============================] - 1s 91us/step - loss: 0.3972 - acc: 0.8347
Epoch 93/100
7200/7200 [==============================] - 1s 116us/step - loss: 0.3974 - acc: 0.8346
Epoch 94/100
7200/7200 [==============================] - 1s 97us/step - loss: 0.3973 - acc: 0.8357
Epoch 95/100
7200/7200 [==============================] - 1s 106us/step - loss: 0.3972 - acc: 0.8344
Epoch 96/100
7200/7200 [==============================] - 1s 80us/step - loss: 0.3970 - acc: 0.8354
Epoch 97/100
7200/7200 [==============================] - 0s 62us/step - loss: 0.3969 - acc: 0.8346
Epoch 98

7200/7200 [==============================] - 1s 90us/step - loss: 0.3996 - acc: 0.8339
Epoch 70/100
7200/7200 [==============================] - 0s 66us/step - loss: 0.3993 - acc: 0.8358
Epoch 71/100
7200/7200 [==============================] - 0s 63us/step - loss: 0.3993 - acc: 0.8361
Epoch 72/100
7200/7200 [==============================] - 0s 66us/step - loss: 0.3994 - acc: 0.8350
Epoch 73/100
7200/7200 [==============================] - 1s 75us/step - loss: 0.3996 - acc: 0.8353
Epoch 74/100
7200/7200 [==============================] - 0s 59us/step - loss: 0.3994 - acc: 0.8351
Epoch 75/100
7200/7200 [==============================] - 1s 75us/step - loss: 0.3991 - acc: 0.8358
Epoch 76/100
7200/7200 [==============================] - 0s 64us/step - loss: 0.3990 - acc: 0.8358
Epoch 77/100
7200/7200 [==============================] - 0s 67us/step - loss: 0.3984 - acc: 0.8353
Epoch 78/100
7200/7200 [==============================] - 0s 65us/step - loss: 0.3993 - acc: 0.8349
Epoch 79/100


Epoch 51/100
7200/7200 [==============================] - 1s 123us/step - loss: 0.3947 - acc: 0.8349
Epoch 52/100
7200/7200 [==============================] - 1s 141us/step - loss: 0.3949 - acc: 0.8354
Epoch 53/100
7200/7200 [==============================] - 1s 164us/step - loss: 0.3943 - acc: 0.8356
Epoch 54/100
7200/7200 [==============================] - 1s 153us/step - loss: 0.3944 - acc: 0.8358
Epoch 55/100
7200/7200 [==============================] - 1s 115us/step - loss: 0.3944 - acc: 0.8357
Epoch 56/100
7200/7200 [==============================] - 1s 120us/step - loss: 0.3944 - acc: 0.8347
Epoch 57/100
7200/7200 [==============================] - 1s 105us/step - loss: 0.3939 - acc: 0.8362
Epoch 58/100
7200/7200 [==============================] - 1s 156us/step - loss: 0.3942 - acc: 0.8335
Epoch 59/100
7200/7200 [==============================] - 1s 129us/step - loss: 0.3942 - acc: 0.8360
Epoch 60/100
7200/7200 [==============================] - 1s 125us/step - loss: 0.3940 - ac

7200/7200 [==============================] - 1s 101us/step - loss: 0.3936 - acc: 0.8378
Epoch 33/100
7200/7200 [==============================] - 1s 135us/step - loss: 0.3934 - acc: 0.8369
Epoch 34/100
7200/7200 [==============================] - 1s 143us/step - loss: 0.3931 - acc: 0.8374
Epoch 35/100
7200/7200 [==============================] - 1s 130us/step - loss: 0.3930 - acc: 0.8368
Epoch 36/100
7200/7200 [==============================] - 1s 162us/step - loss: 0.3927 - acc: 0.8367
Epoch 37/100
7200/7200 [==============================] - 1s 132us/step - loss: 0.3925 - acc: 0.8381
Epoch 38/100
7200/7200 [==============================] - 1s 112us/step - loss: 0.3923 - acc: 0.8372
Epoch 39/100
7200/7200 [==============================] - 1s 142us/step - loss: 0.3924 - acc: 0.8368
Epoch 40/100
7200/7200 [==============================] - 1s 119us/step - loss: 0.3917 - acc: 0.8396
Epoch 41/100
7200/7200 [==============================] - 1s 164us/step - loss: 0.3912 - acc: 0.8382
Epo

KeyboardInterrupt: 